# Analyze accounts
In this step we search for insights about the accounts.

In [1]:
import collections
import json
import os

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import networkx as nx
import pandas as pd
import seaborn as sns
from pandas.plotting import register_matplotlib_converters


register_matplotlib_converters()

%matplotlib inline

plt.style.use("seaborn")

<ipython-input-1-eedb491826c7>:17: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use("seaborn")


In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


# Load the data

In [3]:
data_directory = os.path.join("..", "..", "..", "data")

In [4]:
nome_planilha = 'Maio_2 - 2023.xlsx'
results_file_path = os.path.join(data_directory, "/content/gdrive/MyDrive/Dados_Coletados_Frontrunning_Artigo_SBRC_2024/attacks_052023_2.json")
#results_file_path = os.path.join(data_directory, "/content/drive/MyDrive/DeFi/insertion_results.json")

In [5]:
with open(results_file_path, "r", encoding="utf-8") as results_file:
    results = []
    line = results_file.readline().strip()
    while line != "":
        results.append(json.loads(line))
        line = results_file.readline().strip()

# Collect into data frames

In [6]:
Wei_to_KWei = 1e3
Wei_to_MWei = 1e6
Wei_to_GWei = 1e9
Wei_to_Ether = 1e18

In [7]:
attacker_graph = nx.Graph()

token_attack_count = collections.Counter()
whale_attack_count = collections.Counter()
bot_attack_count = collections.Counter()
interface_attack_count = collections.Counter()

bot_attackers = collections.defaultdict(lambda: set())
bot_exchanges = collections.defaultdict(lambda: set())
bot_profits = collections.defaultdict(lambda: 0.0)

bot_first_blocks = dict()

accounts = set()

block_numbers = []
first_attackers = []
whales = []
second_attackers = []
first_gas_price_deltas = []
second_gas_price_deltas = []
profits = []
costs = []
interfaces = []
exchanges = []
tokens = []
bots = []

for result in results:
    # define some shortcut variables to make code more readable
    first_transaction = result["first_transaction"]
    whale_transaction = result["whale_transaction"]
    second_transaction = result["second_transaction"]

    first_attacker = first_transaction["from"]
    whale = whale_transaction["from"]
    second_ataccker = second_transaction["from"]
    bot = result["bot_address"]

    profit = result["profit_usd"]
    cost = result["cost_usd"]
    exchange = result["exchange_name"]
    token = result["token_name"]
    block_number = result["block_number"]
    interface = result["interface"]

    # initialize first attacker node if needed
    if not attacker_graph.has_node(first_attacker):
        attacker_graph.add_node(first_attacker,
                                first_attacks=0,
                                second_attacks=0,
                                both_attacks=0,
                                exchange_attacks=0,
                                bot_attacks=0,
                                total_profit_usd=0,
                                exchanges=set())
    # initialize second attacker node if needed
    if not attacker_graph.has_node(second_ataccker):
        attacker_graph.add_node(second_ataccker,
                                first_attacks=0,
                                second_attacks=0,
                                both_attacks=0,
                                exchange_attacks=0,
                                bot_attacks=0,
                                total_profit_usd=0,
                                exchanges=set())
    # initialize edge if needed
    if not attacker_graph.has_edge(first_attacker, second_ataccker):
        attacker_graph.add_edge(first_attacker, second_ataccker,
                                exchange_attacks=0,
                                bot_attacks=0,
                                total_profit_usd=0,
                                exchanges=set())

    # exchange interface
    if interface == "exchange":
        # assert first_transaction["to"] == whale_transaction["to"] == second_transaction["to"]
        assert bot is None

        attacker_graph.nodes[first_attacker]["exchange_attacks"] += 1
        if first_attacker != second_ataccker:  # different attackers
            attacker_graph.nodes[second_ataccker]["exchange_attacks"] += 1
        attacker_graph.edges[first_attacker, second_ataccker]["exchange_attacks"] += 1
    # bot interface
    elif interface == "bot":
        assert bot is not None

        bot_attack_count[bot] += 1
        bot_exchanges[bot].add(exchange)
        bot_attackers[bot].add(first_attacker)
        bot_attackers[bot].add(second_ataccker)
        bot_profits[bot] += profit

        if bot not in bot_first_blocks:
            bot_first_blocks[bot] = block_number

        attacker_graph.nodes[first_attacker]["bot_attacks"] += 1
        if first_attacker != second_ataccker:  # different attackers
            attacker_graph.nodes[second_ataccker]["bot_attacks"] += 1
        attacker_graph.edges[first_attacker, second_ataccker]["bot_attacks"] += 1
    # invalid interface
    else:
        raise Exception("Invalid interface.")

    # update first bot node properties
    attacker_graph.nodes[first_attacker]["first_attacks"] += 1
    attacker_graph.nodes[first_attacker]["total_profit_usd"] += profit
    attacker_graph.nodes[first_attacker]["exchanges"].add(exchange)

    # update second bot node properties
    attacker_graph.nodes[second_ataccker]["second_attacks"] += 1
    if first_attacker == second_ataccker:  # same attackers
        attacker_graph.nodes[second_ataccker]["both_attacks"] += 1
    else:  # different attackers
        attacker_graph.nodes[second_ataccker]["total_profit_usd"] += profit
        attacker_graph.nodes[second_ataccker]["exchanges"].add(exchange)

    # update edge properties
    attacker_graph.edges[first_attacker, second_ataccker]["total_profit_usd"] += profit
    attacker_graph.edges[first_attacker, second_ataccker]["exchanges"].add(exchange)

    # update counters outside the graph
    token_attack_count[token] += 1
    whale_attack_count[whale] += 1
    interface_attack_count[interface] += 1

    # update lists outside the graph
    block_numbers.append(block_number)
    first_attackers.append(first_attacker)
    whales.append(whale)
    second_attackers.append(second_ataccker)
    first_gas_price_deltas.append((first_transaction["gasPrice"] - whale_transaction["gasPrice"]) / Wei_to_GWei)
    second_gas_price_deltas.append((whale_transaction["gasPrice"] - second_transaction["gasPrice"]) / Wei_to_GWei)
    profits.append(profit)
    costs.append(cost)
    exchanges.append(exchange)
    interfaces.append(interface)
    tokens.append(token)
    bots.append(bot)

    accounts.add(first_attacker)
    accounts.add(whale)
    accounts.add(second_ataccker)

# some numbers
print("number of exchange attacks", interface_attack_count["exchange"])
print("number of bot attacks", interface_attack_count["bot"])
print("number of attacks", len(results))
print("number of different tokens involved", len(token_attack_count))
print("number of unique attackers involved", len(attacker_graph.nodes))
print("number of unique whale accounts involved", len(whale_attack_count))
print("number of unique accounts involved", len(accounts))

tipos = [
    "Number of Exchange Attacks",
    "Number of Bot Attacks",
    "Number of Attacks",
    "Number of Different Tokens Involved",
    "Number of Unique Attackers Involved",
    "Number of Unique Whale Accounts Involved",
    "Number of Unique Accounts Involved"
]

qtdes = [
    interface_attack_count["exchange"],
    interface_attack_count["bot"],
    len(results),
    len(token_attack_count),
    len(attacker_graph.nodes),
    len(whale_attack_count),
    len(accounts)
]
df_qtde = pd.DataFrame({
    "Tipo": tipos,
    "Qtde.": qtdes
})

number of exchange attacks 392
number of bot attacks 141570
number of attacks 141962
number of different tokens involved 6132
number of unique attackers involved 175
number of unique whale accounts involved 47194
number of unique accounts involved 47358


In [8]:
df_attacks = pd.DataFrame({
    "Block Number": block_numbers,
    "First Attacker": first_attackers,
    "Whale": whales,
    "Second Attacker": second_attackers,
    "First-Whale Gas Price Delta": first_gas_price_deltas,
    "Whale-Second Gas Price Delta": second_gas_price_deltas,
    "Profit": profits,
    "Cost": costs,
    "Interface": interfaces,
    "Exchange": exchanges,
    "Token": tokens,
    "Bot": bots#,
    #"Bot Cluster ID": [bot_hash_to_cluster_id.get(bot) for bot in bots],  # will give None if bot clusters are not updated
})

df_attacks.sample(n=5)

,Block Number,First Attacker,Whale,Second Attacker,First-Whale Gas Price Delta,Whale-Second Gas Price Delta,Profit,Cost,Interface,Exchange,Token,Bot
60159,17307780,0xae2Fc483527B8EF99EB5D9B44875F005ba1FaE13,0xeF09C0a43953a3B80C173E803CEc02e79E71ADb9,0xae2Fc483527B8EF99EB5D9B44875F005ba1FaE13,-0.200,-81.797527,0.054724,21.839551,bot,Uniswap V2,NuNet Utility Token,0x6b75d8AF000000e20B7a7DDf000Ba900b4009A80
94483,17340352,0xb5eaaC5a8649142dcDd13F981767913bcDF7ee42,0xB84dfE3A2f381CbF2E15f882464D5305aaaa3287,0xb5eaaC5a8649142dcDd13F981767913bcDF7ee42,-0.231,-1676.375502,20.188984,275.217150,bot,Uniswap V2,AOLCoin,0xE8c060F8052E07423f71D445277c61AC5138A2e5
50109,17296790,0xae2Fc483527B8EF99EB5D9B44875F005ba1FaE13,0x709e0047D89Dc3f6b1BF413ACb02B9EA266B1a70,0xae2Fc483527B8EF99EB5D9B44875F005ba1FaE13,-25.000,3.224057,-34.161995,51.784752,bot,Uniswap V2,Free For All,0x6b75d8AF000000e20B7a7DDf000Ba900b4009A80
28300,17276596,0xae2Fc483527B8EF99EB5D9B44875F005ba1FaE13,0x3cc28c5C20A157e57E4Cc02D489e16b70a1d9a63,0xae2Fc483527B8EF99EB5D9B44875F005ba1FaE13,-0.100,-495.157730,0.091230,91.159694,bot,Uniswap V2,Do You Like Money?,0x6b75d8AF000000e20B7a7DDf000Ba900b4009A80
32445,17280660,0x76F36d497b51e48A288f03b4C1d7461e92247d5e,0x387875790CDc5F35A40515b5A20879EB8bE1D753,0x76F36d497b51e48A288f03b4C1d7461e92247d5e,-0.100,-1130.831725,-33.537799,319.865995,bot,UniswapV3Pool,Milady,0x4870525EAE23FceB31DF613d179ef6275e1B93a9


In [9]:
df_tokens = pd.DataFrame({
    "Token": token_attack_count.keys(),
    "Attack Count": token_attack_count.values()
})

print("Total: ", len(token_attack_count))

df_tokens.sample(n=20)

Total:  6132


,Token,Attack Count
2988,Twix,3
1688,POB,1
4886,Nexus,3
4085,0xBitcoin Token,2
2423,Tom Cat,10
266,I Made It,2
5968,BADD,7
2053,ASH,17
1315,Inshallah,3
5763,YANG,61


In [10]:
df_whales = pd.DataFrame({
    "Whale Hash": whale_attack_count.keys(),
    "Attack Count": whale_attack_count.values()
})

print("Total: ", len(whale_attack_count))

df_whales.sample(n=20)

Total:  47194


,Whale Hash,Attack Count
14156,0xc2A2b8eb9B41403D649506f571d331252dEC9C8E,18
36149,0x1123865e340cC2ac1f34F937Ecc6c0eBeDFc55ad,1
37210,0xf682672cdDE5ac2557cf6fCFB79DAe6624F3E595,1
44015,0x876aE0EA1166b5f0C93629BDf710c3Ba2118Fe9A,2
36413,0x4EA0d7220B0f149E1994ce2a925fFa9d6D8115E6,2
1601,0xf72A9396Db9af97891f0184AA2Af5c05B4295a5e,3
45242,0x27243DFACd64E698220C963292263268015047Da,1
44248,0xAc1bACC36072959033ca08EcfAf7cD9ed30152EC,1
1688,0xe74B850C92F7F4077Cca91D044af252B38d45BDF,6
29150,0x644AB04B2D084f8Df28900f53b90aD1037f257E6,1


In [11]:
df_attacks[[
    "Cost",
    "Profit",
    "First-Whale Gas Price Delta",
    "Whale-Second Gas Price Delta",
]].describe().style.format("{:.2f}")

,Cost,Profit,First-Whale Gas Price Delta,Whale-Second Gas Price Delta
count,141962.00,141962.00,141962.00,141962.00
mean,170.62,993.50,-5.21,-394.99
std,352.05,86026.50,37.42,1435.88
min,6.24,-449593.56,-3500.00,-162847.61
25%,36.02,-58.37,-2.00,-377.05
50%,78.47,-10.65,-0.30,-152.87
75%,176.70,0.30,-0.10,-49.76
max,22357.49,13412389.81,3447.56,2000.00


In [12]:
!pip install xlsxwriter

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.8/154.8 kB 1.5 MB/s eta 0:00:00


In [13]:
with pd.ExcelWriter(nome_planilha, engine="xlsxwriter") as writer:
  df_qtde.to_excel(writer, sheet_name='Qtde. analisada', encoding="utf8")
  df_attacks.to_excel(writer, sheet_name='Ataques')
  df_tokens.to_excel(writer, sheet_name='Tokens')
  df_whales.to_excel(writer, sheet_name='Whales')
  df_attacks[[
    "Cost",
    "Profit",
    "First-Whale Gas Price Delta",
    "Whale-Second Gas Price Delta",
  ]].describe().to_excel(writer, sheet_name='Resultados')

/usr/local/lib/python3.10/dist-packages/pandas/util/_decorators.py:211: FutureWarning: the 'encoding' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'encoding'
  return func(*args, **kwargs)
